# Backtest and Optimiser !!!
1. This script can backtest and optimise any single position strategy.
2. The script uses a csv from Binance spot BTC/USDT, please ensure you have a suitable CSV before attempting to read in a dataframe.
3. When testing/optimising your strategy, it is a good idea to mask the data to a smaller sample set to confirm accuracy before completing a full backtest. As a rough example to optimise 5 timeframes, with two moving averages, and four fixed TP and SL options i.e 740 iterations of Backtest() could take over 8 hours!

In [50]:
#Libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import itertools
#import plotly.graph_objects as go

In [51]:
#READING AND CLEANING + SET INDEX
df = pd.read_csv("../Binance_BTCUSDT_1min.csv")
df = df.iloc[:,:6]
df.columns=['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.reset_index(drop=True, inplace=True)
df.Date = pd.to_datetime(df.Date)
df = df.set_index("Date")
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796
...,...,...,...,...,...
2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.557760
2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.890100
2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.087330


In [53]:
#MASKING TO DECREASE SAMPLE SIZE
start_date = "2020-01-01 00:00:00"
end_date = "2024-01-01 00:00:00"
mask = (df.index > start_date) & (df.index <= end_date)
df=df.loc[mask]
df

,Open,High,Low,Close,Volume
Date,,,,,
2020-01-01 00:01:00,7187.67,7188.06,7182.20,7184.03,7.248148
2020-01-01 00:02:00,7184.41,7184.71,7180.26,7182.43,11.681677
2020-01-01 00:03:00,7183.83,7188.94,7182.49,7185.94,10.025391
2020-01-01 00:04:00,7185.54,7185.54,7178.64,7179.78,14.911105
2020-01-01 00:05:00,7179.76,7182.51,7178.20,7179.99,12.463243
...,...,...,...,...,...
2023-06-13 16:46:00,25848.59,25852.45,25843.93,25850.01,87.557760
2023-06-13 16:47:00,25850.00,25866.98,25850.00,25866.97,50.890100
2023-06-13 16:48:00,25866.98,25866.98,25855.67,25863.11,53.087330


In [54]:
#Function for resampling the dataframe
def resample_df(df, freq):
    resampled_open = df.Open.resample(freq).first()
    resampled_high = df.High.resample(freq).max()
    resampled_low = df.Low.resample(freq).min()
    resampled_close = df.Close.resample(freq).last()
    resampled_volume = df.Volume.resample(freq).sum()
    new_df = pd.concat([resampled_open, resampled_high, resampled_low, resampled_close, resampled_volume], axis=1)
    new_df.dropna(inplace=True)
    return new_df

In [55]:
def calc_ma(new_df, n, m):
    new_df['SMA50'] = new_df.Close.rolling(n).mean()
    new_df['SMA200'] = new_df.Close.rolling(m).mean()

In [56]:
def calc_price(new_df):
    new_df["price"] = new_df.Open.shift(-1)

In [73]:
def calc_buy_signal(new_df):
    new_df["buy_signal"] = np.where((new_df.SMA50>new_df.SMA200 & new_df.SMA50[1]>new_df.SMA200[1]), True, False)

In [74]:
new_df = resample_df(df,"1D")

In [75]:
calc_ma(new_df, 50, 200)

In [76]:
calc_buy_signal(new_df)

In [77]:
new_df[new_df.buy_signal>0]

,Open,High,Low,Close,Volume,SMA50,SMA200,buy_signal
Date,,,,,,,,
2020-07-18,9154.31,9219.30,9121.10,9170.28,22554.541457,9388.6278,8526.80180,True
2020-07-19,9170.30,9232.27,9101.35,9208.99,26052.019417,9378.8532,8536.84250,True
2020-07-20,9208.99,9221.52,9131.00,9160.78,35458.764082,9373.1034,8547.81785,True
2020-07-21,9160.78,9437.73,9152.80,9390.00,60413.582486,9356.8880,8558.04305,True
2020-07-22,9390.00,9544.00,9261.00,9518.16,48815.004107,9356.8904,8568.86330,True
...,...,...,...,...,...,...,...,...
2023-06-09,26498.62,26783.33,26269.91,26477.81,27934.709700,27533.8894,23442.56090,True
2023-06-10,26477.80,26533.87,25358.00,25841.21,64944.601080,27505.4568,23490.63225,True
2023-06-11,25841.22,26206.88,25634.70,25925.55,30014.295950,27467.6308,23537.24445,True


In [64]:
new_df.dropna(inplace=True)

In [ ]:
#ADDING COLUMNS AND SIGNALS
#adx = ta.adx(df.High, df.Low, df.Close, length=9)
#df['ADX'] = adx.ADX_9
#df['RSI'] = ta.rsi(df.Close, length=9)
#df['CCI'] = ta.cci(df.High, df.Low, df.Close, length=9)
#df['VWAP'] = ta.vwap(df.High, df.Low, df.Close, df.Volume, anchor="M")
#df["SMA_200"] = ta.sma(df.Close, 200)
#df["SMA_50"] = ta.sma(df.Close, 50)
#df["lookBackLow"] = df.Low.rolling(2).min()
#df.dropna(inplace=True)
#df

#calc_indicators functions
#def calc_vwap(new_df, anchor_period):
    #new_df['VWAP'] = ta.vwap(new_df.High, new_df.Low, new_df.Close, new_df.Volume, anchor=anchor_period)

#def calc_vwap(new_df, anchor_period):
    #new_df['VWAP'] = ta.vwap(new_df.High, new_df.Low, new_df.Close, new_df.Volume, anchor=anchor_period)
    
#def calc_ma(new_df, n):
    #new_df['SMA'] = new_df.Close.rolling(n).mean()
    
#def calc_fast_rsi(new_df, frsi):
    #new_df["fast_rsi"] = ta.rsi(new_df.Close, length=frsi)
    
    #TO RUN CALC
    #calc_vwap(new_df, anchor_period)
    #calc_fast_ma(new_df, m)
    #calc_slow_ma(new_df,n)

#def calc_pivots(new_df, lb):
    #new_df["min_val"] = df.Low.rolling(lb).min()
    #new_df["max_val"] = df.High.rolling(lb).max()

#def calc_rsi(new_df, rsi_length, rsiSMA_period):
    #new_df["rsi"] = ta.rsi(new_df.Close, length=rsi_length).round(2)
    #new_df["rsiSMA"] = new_df.rsi.rolling(rsiSMA_period).mean().round(2)
    #new_df['rsi_std'] = new_df.rsi.rolling(20).std(ddof=0)*2
    #new_df["rsibbUpper"] = (new_df.rsiSMA + new_df.rsi_std).round(2)
    #new_df["rsibbLower"] = (new_df.rsiSMA - new_df.rsi_std).round(2)

In [ ]:
overSold = 0
overBought = 0

In [ ]:
def backtest(df, freq, rsi_length, rsiSMA_period, overSold, tp, sl):

    new_df = resample_df(df, freq)
    calc_rsi(new_df, rsi_length, rsiSMA_period)
    calc_price(new_df)
    calc_buy_signal(new_df, overSold)
    new_df.dropna(inplace=True)
    
    in_position = False
    equity = 1000
    leverage = 10
    trades = []
    current_trade = {}
    
    for i in range(len(new_df)-1):
        #Check exit conditions
        if in_position:
            if new_df.iloc[i].Low < current_trade["sl_price"]:
                current_trade["exit_price"] = current_trade["sl_price"]
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["exit_price"],
                    "pnl":(current_trade["exit_price"]-current_trade["entry_price"])/current_trade["exit_price"],
                })
                current_trade = {}
                in_position = False
                
            elif new_df.iloc[i].High > current_trade["tp_price"]:
                current_trade["exit_price"] = current_trade["tp_price"]
                trades.append({
                    "entry_time":current_trade["entry_time"],
                    "entry_price":current_trade["entry_price"],
                    "tp_target":current_trade["tp_price"],
                    "sl_target":current_trade["sl_price"],
                    "exit_time":new_df.iloc[i].name,
                    "exit_price":current_trade["tp_price"],
                    "pnl":(current_trade["exit_price"]/current_trade["entry_price"])-1,
                })
                current_trade = {}
                in_position = False
        
        #Check entry conditions
        if not in_position:
            if new_df.iloc[i].buy_signal == True:
                current_trade["entry_price"] = new_df.iloc[i].price
                current_trade["entry_time"] = new_df.iloc[i+1].name
                current_trade["tp_price"] = new_df.iloc[i].price*tp
                current_trade["sl_price"] = new_df.iloc[i].price*sl
                in_position = True
                
    data = pd.DataFrame(trades)
    amount = len(data)
    winrate = len(data.loc[data.pnl.values>0])/len(data)*100
    pnl = sum(pd.Series(data.pnl))
    return amount, winrate, pnl
    #return pd.DataFrame(trades) 

In [ ]:
#backtest(df, freq, rsi_length, rsiSMA_period, overSold, tp, sl):
#1H	7	20	20	1.02	0.97

In [ ]:
backtest(df, "1D", 2, 20, 20, 1.02, 0.97)

# Optimiser

In [ ]:
# Define the parameter combinations
freq_values = ["1H", "4H", "1D"]
#anchor_period_values = ["D", "W"]
rsiSMA_values = [20, 50, 100, 200]
rsi_values = [2,3]
overBought_values = [70, 80]
overSold_values = [20, 30]
tp_values = [1.03, 1.04, 1.12]
sl_values = [0.98, 0.97, 0.96]

In [ ]:
#backtest(df, freq, rsi_length, rsiSMA_period, overSold, tp, sl):

In [ ]:
#Create an empty dataframe
results_df = pd.DataFrame(columns=["freq", "rsi_length", "sma_period", "overSold", "tp", "sl", "amount", "winrate", "pnl"])

In [ ]:
for freq, rsi_length, rsiSMA_period, overSold, tp, sl in itertools.product(freq_values, rsi_values, rsiSMA_values, overSold_values, tp_values, sl_values):
    amount, winrate, pnl = backtest(df, freq, rsi_length, rsiSMA_period, overSold, tp, sl)
    result = pd.DataFrame([[freq, rsi_length, rsiSMA_period, overSold, tp, sl, amount, winrate, pnl]], columns=["freq", "rsi_length", "sma_period", "overSold", "tp", "sl", "amount", "winrate", "pnl"])
    results_df = pd.concat([results_df, result], ignore_index=True)

# Print the results DataFrame
print(results_df)

In [ ]:
results_df

In [ ]:
#results_df[results_df.pnl.max()]
results_df["pnl1"] = results_df.pnl.round(2)

In [ ]:
filtered_results = results_df[(results_df["winrate"] > 50) & (results_df["amount"] > 300) & (results_df["pnl1"] > 1.5) & (results_df["sl"]>0.95)]
filtered_results

In [ ]:
results_df[results_df.pnl>3]

In [ ]:
#results_df.to_csv('BTCUSDT_RSIBB_BREAKOUT-Backtest.csv', index=True)

In [ ]:
pd.read_csv('BTCUSDT_RSIBB_BREAKOUT-Backtest.csv')

# Visualisation

In [ ]:
#CANDLE STICK AND MERGED PLOTS
fig = go.Figure(data=[go.Candlestick(
    x = new_df.index,
    open = new_df.Open,
    high = new_df.High,
    low = new_df.Low,
    close = new_df.Close,
    increasing_line_color = "rgba(0, 0, 0, 0.5)",
    decreasing_line_color = "rgba(0, 0, 0, 0.5)",
    name="Candlesticks"
    ),
    go.Scatter(x=new_df.index, y=new_df.VWAP, line=dict(color='red', width=1)),
    go.Scatter(x=new_df.index, y=new_df.SMA, line=dict(color='blue', width=1))
])

#ENTRY PLOTS
fig.add_trace(go.Scatter(
    x=trades.entry_time,
    y=trades.entry_price,
    mode = "markers",
    customdata=trades,
    marker_symbol="triangle-up",
    marker_color="rgba(0, 255, 0, 0.9)",
    marker_line_color="rgba(0, 0, 0, 0.5)",
    marker_size =13,
    marker_line_width=1,
    name="Entries",
    hovertemplate="Entry Time: %{customdata[1]}<br>"
        "Entry Price: %{y:.2f}<br>"
        "Size: %{customdata[2]:.6f}"
))

#EXIT PLOTS
fig.add_trace(go.Scatter(
    x=trades.exit_time,
    y=trades.exit_price,
    mode = "markers",
    customdata=trades,
    marker_symbol="triangle-down",
    marker_color="rgba(255, 0, 0, 0.9)",
    marker_line_color="rgba(0, 0, 0, 0.5)",
    marker_size =13,
    marker_line_width=1,
    name="Exits",
    hovertemplate="Exit Time: %{customdata[4]}<br>"
        "Exit Price: %{y:.2f}<br>"
        #"Size: %{customdata[2]:.6f}"
))

#fig.update_layout (xaxis_rangeslider_visable = False)
fig